In [35]:
# load necessary packages
using CSV, DataFrames; rd, wr = CSV.read, CSV.write
using JSON, GeoJSON, DataStructures
using StatsBase, NaNMath; nm = NaNMath

NaNMath

In [57]:
ctrax = rd("../../data/geo/ct2010.csv") |> DataFrame; tag = :BoroCT2010
fails = rd("../python/failed.csv") |> DataFrame; fs = fails[:,:failed]
T = ctrax[:,tag]; n = length(T); avgs = OrderedDict()
;

In [58]:
DIR = "../python/jsons/"; Fs = Int[]
for i ∈ 1:n
    t = T[i];
    try
        f = JSON.Parser.parsefile("$DIR$t.json")
        pull = f["results"][1]; locs = pull["locations"]
        time = [loc["properties"][1]["travel_time"] for loc in locs]
        time = convert(Array{Float64,1},time)
        if "unreachable" in keys(pull)
            push!(time,[NaN for i ∈ length(pull["unreachable"])]...)
        end
        avgs[t] = nm.mean(time./60)
    catch
        @warn "$t not found..."
        avgs[t] = NaN
        push!(Fs,t)
    end
end
insertcols!(ctrax,ncol(ctrax)+1,:travel_time => [v for v in values(avgs)])

┌ Warning: 3070203 not found...
└ @ Main In[58]:14
┌ Warning: 5990100 not found...
└ @ Main In[58]:14
┌ Warning: 4107202 not found...
└ @ Main In[58]:14


,BoroCode,the_geom
,Int64,String
1,5,"MULTIPOLYGON (((-74.07920577013245 40.64343078374567, -74.07913504647637 40.64333789954809, -74.07819773831595 40.64211481672276, -74.07884632409665 40.641995568528166, -74.0791223401088 40.64194431928626, -74.07965266950632 40.641855351552806, -74.08055489346589 40.64169150377349, -74.08148001617707 40.64155046162583, -74.08130714145031 40.64074191709323, -74.08110049694088 40.639922825339106, -74.08191207797016 40.6398059117602, -74.08194326513642 40.63993327788856, -74.0820464807701 40.640338526958104, -74.08228199570526 40.640600896453016, -74.08503312179074 40.64019785806871, -74.0872057798552 40.639895367647135, -74.08709163679771 40.64033437636213, -74.08701899114665 40.640613790850445, -74.08682767812078 40.64133802397936, -74.08667746827268 40.64190963327233, -74.08652380457303 40.64256408357222, -74.08575006381034 40.642846120676055, -74.08496857468032 40.643130075816266, -74.08414352403344 40.64343089730628, -74.0820726845645 40.64420760543788, -74.08237118148979 40.64464507998435, -74.08028198256949 40.64479321566102, -74.07920577013245 40.64343078374567)))"
2,1,"MULTIPOLYGON (((-73.96432543478758 40.756381530990915, -73.96478633130418 40.755755731448126, -73.96524094428382 40.75512992861379, -73.9656961990018 40.754505070476604, -73.96615683421602 40.753877851122866, -73.96653673268192 40.753350256126176, -73.96661581699009 40.75324342594179, -73.96897803741389 40.75425053544143, -73.97121704465083 40.75519332425508, -73.97075919418829 40.755824851242494, -73.97031393611448 40.7564530190295, -73.96984859921461 40.75707595800298, -73.96939411311261 40.757699528158426, -73.96893510322617 40.75833009163121, -73.96670187508813 40.75738080380005, -73.96432543478758 40.756381530990915)))"
3,1,"MULTIPOLYGON (((-73.96802436915851 40.759578140052824, -73.96847979621307 40.75895012807789, -73.96893510322617 40.75833009163121, -73.96939411311261 40.757699528158426, -73.96984859921461 40.75707595800298, -73.97031393611448 40.7564530190295, -73.97075919418829 40.755824851242494, -73.97121704465083 40.75519332425508, -73.97281967445284 40.75587055150212, -73.97365314711985 40.756223589840275, -73.97432789790405 40.75650959960279, -73.97443866383502 40.7565563063462, -73.97398183196347 40.75717975808582, -73.97352419611919 40.75780844027898, -73.97306851954798 40.758431494196365, -73.97261271418446 40.75905930848452, -73.97215595488032 40.75968124035725, -73.9716960921487 40.76031266144978, -73.97124277307127 40.76093641847906, -73.97113517599529 40.760890852516724, -73.96962500126692 40.760251440039696, -73.96802436915851 40.759578140052824)))"
4,1,"MULTIPOLYGON (((-73.97124277307127 40.76093641847906, -73.9716960921487 40.76031266144978, -73.97215595488032 40.75968124035725, -73.97261271418446 40.75905930848452, -73.97306851954798 40.758431494196365, -73.97352419611919 40.75780844027898, -73.97398183196347 40.75717975808582, -73.97443866383502 40.7565563063462, -73.97453865562939 40.75659708520902, -73.97604542132363 40.75723205763824, -73.9776548272757 40.75791142377997, -73.97720026523596 40.75853793354625, -73.97674467602127 40.759164737776686, -73.97628870099064 40.75978674916537, -73.97583011985374 40.76041852021435, -73.97537360664667 40.761039907074164, -73.97491476221624 40.76166727005669, -73.97445730550224 40.76229308352487, -73.97285250503253 40.76161435775506, -73.97133777276214 40.76097729818959, -73.97124277307127 40.76093641847906)))"
5,1,"MULTIPOLYGON (((-73.97445730550224 40.76229308352487, -73.97491476221624 40.76166727005669, -73.97537360664667 40.761039907074164, -73.97583011985374 40.76041852021435, -73.97628870099064 40.75978674916537, -73.97674467602127 40.759164737776686, -73.97720026523596 40.75853793354625, -73.9776548272757 40.75791142377997, -73.97891172591584 40.75843653883477, -73.98088574583751 40.75927066953552, -73.98042606433056 40.75989908107926, -73.9799718818968 40.760524701377975, -73.97951319160578 40.7611485618501, -73.97905731164806 40.7617707197265

In [60]:
ctrax |> wr("../../data/info/cttimes.csv")

"../../data/info/cttimes.csv"

In [ ]:
jtrax = JSON.Parser.parsefile("data/geo/ct2010.geojson")

In [ ]:
FT = jtrax["features"]; k = length(FT)
for i ∈ 1:k
	code = FT[i]["properties"]["boro_ct2010"]; code = Base.parse(Int,code)
	!(code in Fs) ? ttime = avgs[code] : ttime = NaN
    jtrax["features"][i]["properties"]["travel_time"] = ttime
end

In [ ]:
open("data/geo/cttimes.geojson", "w") do f
	write(f, JSON.json(jtrax))
end